In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data63881


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [3]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
# !pip install beautifulsoup4 -t /home/aistudio/external-libraries
!pip install lightgbm==2.3.1 -t /home/aistudio/external-libraries
# !pip install catboost==0.23 -t /home/aistudio/external-libraries
# !pip install jieba -t /home/aistudio/external-libraries
# !pip install gensim -t /home/aistudio/external-libraries
!pip uninstall --yes pandas
!pip install pandas==1.0.5 -t /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 1.2MB 12.6MB/s eta 0:00:01
     |████████████████████████████████| 14.5MB 9.3MB/s eta 0:00:011
     |████████████████████████████████| 25.9MB 8.6MB/s eta 0:00:012    |█████████                       | 7.4MB 13.7MB/s eta 0:00:02     |████████████████████            | 16.1MB 8.6MB/s eta 0:00:02
     |████████████████████████████████| 6.8MB 21.0MB/s eta 0:00:01
     |████████████████████████████████| 307kB 18.0MB/s eta 0:00:01
ERROR: seaborn 0.10.0 requires pandas>=0.22.0, which is not installed.
ERROR: paddlehub 1.6.0 requires pandas; python_version >= "3", which is not installed.
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 10.1MB 15.0MB/s eta 0:00:01    |█████████████▌                  | 4.2MB 15.0MB/s eta 0:00:01
     |████████████████████████████████| 235kB

In [4]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [5]:
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [6]:
train_transaction = pd.read_csv('data/data63881/train_transaction.csv')
test_transaction = pd.read_csv('data/data63881/test_transaction.csv')
train_identity = pd.read_csv('data/data63881/train_identity.csv')
test_identity = pd.read_csv('data/data63881/test_identity.csv')

In [7]:
train = train_transaction.merge(train_identity, on='TransactionID', how='left')
test = test_transaction.merge(test_identity, on='TransactionID', how='left')
data = pd.concat([train, test], axis=0, ignore_index=True)
del train, test
gc.collect()

0

In [8]:
C = ['C{}'.format(i) for i in range(1, 15)]
data['C_sum'] = data[C].sum(1)

D = ['D{}'.format(i) for i in range(1, 16)]
data['D_sum'] = data[D].sum(1)

V = ['V{}'.format(i) for i in range(1, 340)]
data['V_sum'] = data[V].sum(1)

In [9]:
data['P_emaildomain_R_emaildomain'] = data['P_emaildomain'].astype(str) + data['R_emaildomain'].astype(str)
data['addr1_addr2'] = data['addr1'].astype(str) + data['addr2'].astype(str)

object_cols = ['ProductCD', 'card4', 'card6', 'DeviceType', 'DeviceInfo', 'P_emaildomain', 'R_emaildomain',
               'P_emaildomain_R_emaildomain', 'addr1_addr2']
M_cols = ['M{}'.format(i) for i in range(1, 10)]
id_cols = ['id_12', 'id_16', 'id_27', 'id_28', 'id_29', 'id_35', 'id_36', 'id_37', 'id_38', 'id_15',
           'id_23', 'id_34', 'id_30', 'id_31', 'id_33']
cat_cols = object_cols + M_cols + id_cols

In [10]:
for i in cat_cols:
    le = LabelEncoder()
    data[i] = le.fit_transform(data[i].astype(str))
    data[i] = data[i].astype('category')

for i in ['addr1', 'addr2']:
    data[i] = data[i].astype('category')

In [11]:
train = data[data['isFraud'].notnull()]
test = data[data['isFraud'].isnull()]

y = train['isFraud']
train.drop(['isFraud', 'TransactionID'], axis=1, inplace=True)
test.drop(['isFraud', 'TransactionID'], axis=1, inplace=True)
used_cols = train.columns
test = test[used_cols]

X_train, X_valid, y_train, y_valid = train_test_split(train, y, random_state=2020)

In [12]:
# def auc_select(X_train, y_train, X_valid, y_valid, cols, threshold=0.52):
#     useful_dict = dict()
#     useless_dict = dict()
#     params = {
#         'objective': 'binary',
#         'boosting': 'gbdt',
#         'metric': 'auc',
#         'learning_rate': 0.1,
#         'num_leaves': 31,
#         'lambda_l1': 0,
#         'lambda_l2': 1,
#         'num_threads': 23,
#         'min_data_in_leaf': 20,
#         'first_metric_only': True,
#         'is_unbalance': True,
#         'max_depth': -1,
#         'seed': 2020
#     }
#     for i in cols:
#         print(i)
#         try:
#             lgb_train = lgb.Dataset(X_train[[i]].values, y_train)
#             lgb_valid = lgb.Dataset(X_valid[[i]].values, y_valid, reference=lgb_train)
#             lgb_model = lgb.train(
#                 params,
#                 lgb_train,
#                 valid_sets=[lgb_valid, lgb_train],
#                 num_boost_round=1000,
#                 early_stopping_rounds=50,
#                 verbose_eval=500
#             )
#             print('*' * 10)
#             print(lgb_model.best_score['valid_0']['auc'])
#             if lgb_model.best_score['valid_0']['auc'] > threshold:
#                 useful_dict[i] = lgb_model.best_score['valid_0']['auc']
#             else:
#                 useless_dict[i] = lgb_model.best_score['valid_0']['auc']
#         except:
#             print('Error: ', i)
#     useful_cols = list(useful_dict.keys())
#     useless_cols = list(useless_dict.keys())
#     return useful_dict, useless_dict, useful_cols, useless_cols


# useful_dict, useless_dict, useful_cols, useless_cols = auc_select(X_train, y_train, X_valid, y_valid, used_cols, threshold=0.52)
# print(useless_cols)

In [13]:
auc_to_drop = ['card4', 'C3', 'V41', 'V98', 'V100', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V113', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V135', 'V138', 'V141', 'V142', 'V161', 'V162', 'V163', 'V286', 'V297', 'V299', 'V300', 'V301', 'V305', 'V311', 'V319', 'V325', 'V326', 'V334', 'id_07', 'id_08', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_32', 'id_34']

X_train.drop(auc_to_drop, axis=1, inplace=True)
X_valid.drop(auc_to_drop, axis=1, inplace=True)
train.drop(auc_to_drop, axis=1, inplace=True)
test.drop(auc_to_drop, axis=1, inplace=True)
gc.collect()

40

In [14]:
%%time
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid, reference=dtrain)
params = {
    'objective': 'binary',
    'boosting': 'gbdt',
    'metric': 'auc',
    # 'metric': 'None',  # 用自定义评估函数是将metric设置为'None'
    'learning_rate': 0.1,
    'num_leaves': 31,
    'lambda_l1': 0,
    'lambda_l2': 1,
    'num_threads': 23,
    'min_data_in_leaf': 20,
    'first_metric_only': True,
    'is_unbalance': True,
    'max_depth': -1,
    'seed': 2020
}
valid_model = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dvalid],
    num_boost_round=1000000,
    early_stopping_rounds=200,
    verbose_eval=300
)

importance = valid_model.feature_importance(importance_type='gain')
feature_name = valid_model.feature_name()
df_importance = pd.DataFrame({
    'feature_name': feature_name,
    'importance': importance
}).sort_values(by='importance', ascending=False)
df_importance['normalized_importance'] = df_importance['importance'] / df_importance['importance'].sum()
df_importance['cumulative_importance'] = np.cumsum(df_importance['normalized_importance'])
record_low_importance = df_importance[df_importance['cumulative_importance'] > 0.95]
to_drop = list(record_low_importance['feature_name'])
print(to_drop)
df_importance.to_csv('sub/imp_{}_2.csv'.format(time.strftime('%Y%m%d')), index=False)

Training until validation scores don't improve for 200 rounds
[300]	training's auc: 0.989606	valid_1's auc: 0.954804
[600]	training's auc: 0.99709	valid_1's auc: 0.961037
[900]	training's auc: 0.999049	valid_1's auc: 0.963161
[1200]	training's auc: 0.999668	valid_1's auc: 0.963748
Early stopping, best iteration is:
[1234]	training's auc: 0.999703	valid_1's auc: 0.963877
Evaluated only: auc
['id_14', 'V94', 'V25', 'id_18', 'V321', 'V4', 'V69', 'V77', 'V160', 'V257', 'V81', 'V11', 'V10', 'V306', 'V96', 'V215', 'D7', 'id_11', 'V129', 'V66', 'V6', 'V267', 'V124', 'V74', 'V206', 'C4', 'V207', 'V26', 'V73', 'V133', 'V217', 'V292', 'V137', 'V289', 'V99', 'V30', 'V143', 'id_04', 'C7', 'id_15', 'V339', 'V316', 'V8', 'V3', 'V265', 'V7', 'V251', 'V261', 'V221', 'V318', 'V271', 'V322', 'V166', 'V264', 'V270', 'V132', 'V47', 'V2', 'V85', 'V209', 'V224', 'V338', 'V40', 'V288', 'V277', 'V234', 'V178', 'V253', 'V290', 'V260', 'V205', 'V276', 'V46', 'V60', 'V52', 'V274', 'V293', 'V188', 'V329', 'V220',

In [15]:
X_train.drop(to_drop, axis=1, inplace=True)
X_valid.drop(to_drop, axis=1, inplace=True)
train.drop(to_drop, axis=1, inplace=True)
test.drop(to_drop, axis=1, inplace=True)
gc.collect()

216

In [16]:
%%time
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid, reference=dtrain)
dall = lgb.Dataset(train, y)

valid_model2 = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dvalid],
    num_boost_round=1000000,
    early_stopping_rounds=200,
    verbose_eval=300
)

Training until validation scores don't improve for 200 rounds
[300]	training's auc: 0.989964	valid_1's auc: 0.954877
[600]	training's auc: 0.997098	valid_1's auc: 0.960662
[900]	training's auc: 0.999041	valid_1's auc: 0.963168
[1200]	training's auc: 0.999667	valid_1's auc: 0.964254
[1500]	training's auc: 0.999888	valid_1's auc: 0.964813
[1800]	training's auc: 0.999964	valid_1's auc: 0.965323
[2100]	training's auc: 0.999988	valid_1's auc: 0.96579
[2400]	training's auc: 0.999996	valid_1's auc: 0.966189
Early stopping, best iteration is:
[2742]	training's auc: 0.999999	valid_1's auc: 0.966717
Evaluated only: auc
CPU times: user 1h 5min 19s, sys: 10.6 s, total: 1h 5min 30s
Wall time: 16min 30s


In [17]:
%%time
model = lgb.train(
    params,
    dall,
    num_boost_round=valid_model2.best_iteration
)

CPU times: user 39min 1s, sys: 6.62 s, total: 39min 8s
Wall time: 9min 53s


In [18]:
%%time
pred = model.predict(test)
sub = pd.DataFrame({'id': range(len(test))})
sub['isFraud'] = pred
sub.to_csv('./sub/baseline_{}_02.csv'.format(time.strftime('%Y%m%d')), index=False, header=None)

CPU times: user 48.8 s, sys: 574 ms, total: 49.4 s
Wall time: 13.3 s
